# Distribute Data for Crossvalidation

## Things To Do:-

<ol>
    <li>Part 1</li>
    <li>Extract Zip File, Parent Folder would contain Train and Test folder</li>
    <li>Create a new folder crossvalidation</li>
    <li>Train Folder contains 46 subfolders and 1700 Images, Extract 25% of images from the train folder to crossvalidation</li>
    <li>Part 2</li>
    <li>Create a Dataframe from all th files in the Train, Each Image is 32 * 32, we need to create a row vector 1 * 1024 for each row, </li>
    <li>Add a numbered label to decide the category in which the row falls</li>
    <li>So final row would be 1 * 1025 and rows would be 46*1275, 46 folders and 1275 files in each</li>
    <li>Final size of Dataframe would be 58650 * 1025</li>
    <li>To final test the accuracy random pick any row of dataframe and check the image is of correct label</li>
</ol>



In [1]:
import pandas as pd
import numpy as np

from zipfile import ZipFile 

import os.path
from os import path
import shutil

In [2]:
# specifying the zip file name 
zip_file_name = "data/DevanagariHandwrittenCharacterDataset.zip"

zip_folder_name = "data/DevanagariHandwrittenCharacterDataset"

train_folder_name = "data/DevanagariHandwrittenCharacterDataset/Train"

crossvalidation_folder_name = "data/DevanagariHandwrittenCharacterDataset/crossvalidation"

zip_extract_location = "data/"

percentage_of_files_to_extract = .75

sample_crossvalidation_file = crossvalidation_folder_name + "/character_1_ka/1340.png"

In [3]:
# Extract Zip file if we haven't extracted it yet

# Commented for safety
#if path.exists(zip_folder_name):
#    shutil.rmtree(zip_folder_name)

if not path.exists(zip_folder_name):
    
    # opening the zip file in READ mode 
    with ZipFile(zip_file_name, 'r') as zip: 
        
        # printing all the contents of the zip file 
        #zip.printdir() 

        # extracting all the files 
        print('Extracting all the files now...') 
        zip.extractall(path=zip_extract_location) 
        print('Done!') 
else:
    print(zip_folder_name + " Exists, Already Extracted")


data/DevanagariHandwrittenCharacterDataset Exists, Already Extracted


In [4]:
print(os.getcwd())

/Users/agaxmp04/Documents/JupyterNotebook/Jaskerat_ML_Course


In [5]:
# define the name of the Crossvalidation directory to be created
crossvalidationpath = os.getcwd() + "/" + zip_folder_name + "/" + "crossvalidation"

try:
    os.mkdir(crossvalidationpath)
except OSError:
    # if the folder is already there, it would fail
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

Creation of the directory <module 'posixpath' from '/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/posixpath.py'> failed


In [6]:
# Move 75% of files from Train to Crossvalidation folder

if not path.exists(sample_crossvalidation_file):
    
    for dirpath, dirnames, filenames in os.walk(train_folder_name):
        if len(filenames) != 0:
            #print(dirpath)
            #print(dirnames)
            #print(filenames)
            destination = crossvalidation_folder_name + "/" + str(dirpath.split(sep='/').pop())

            if not path.exists(destination):
                os.mkdir(destination)

            files_to_be_moved = int(len(filenames) * percentage_of_files_to_extract)
            #print(destination)
            #print(files_to_be_moved)
            for i in range(files_to_be_moved):
                src = dirpath + "/" + filenames[i]
                dest = destination + "/" + filenames[i]
                shutil.move(src = src, dst = destination);
else:
    print("Already moved")


Already moved


# Part 2

In [7]:
import imageio
import visvis as vv

In [8]:
im = imageio.imread(str(sample_crossvalidation_file))

In [9]:
im

Array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0, 114, ..., 230,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)

In [10]:
imr = im.reshape((1,1024))

In [11]:
print(imr.shape)

np_im = np.array(imr)
im_im2 = np.append(np_im, [[34]], axis=1)
print(im_im2[:,:-1].shape)
print(im_im2[:,:-1].reshape((32,32)))
im_im3 = im_im2[:,:-1].reshape((32,32))

# which category added
print(im_im2[:,-1:][0][0])

(1, 1024)
(1, 1024)
[[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0 114 ... 230   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
34


In [12]:
def getImageFromImgCat(np_image):
    return np_image[:,:-1].reshape((32,32))

In [13]:
def getCategoryFromImgCat(np_image):
    return np_image[:,-1:][0][0]

In [14]:
sample_crossvalidation_file.split("/").pop(-2)

'character_1_ka'

In [23]:
cv_df = pd.DataFrame(columns=['Image_Category', 'Category'])

In [24]:
count = 0
for dirpath, dirnames, filenames in os.walk(train_folder_name):
    #print(dirpath)
    #print(len(filenames))
    if len(filenames) != 0:
        for i in range(len(filenames)):
        #for i in range(2):
            name = filenames[i]
            directory_name = dirpath.split('/')[-1]
            image_path = src = dirpath + "/" + name
            image = imageio.imread(str(image_path))
            image = image.reshape((1,1024))
            np_image = np.array(image)
            np_image = np.append(np_image, [[count]], axis=1)
            #cv_df.loc[i + count*2] = [np_image, directory_name]
            cv_df = cv_df.append({'Image_Category':np_image, 'Category':directory_name}, ignore_index=True)
            #print(directory_name)
            #print(image_path)
        count += 1

data/DevanagariHandwrittenCharacterDataset/Train
0
data/DevanagariHandwrittenCharacterDataset/Train/character_10_yna
425
data/DevanagariHandwrittenCharacterDataset/Train/character_11_taamatar
425
data/DevanagariHandwrittenCharacterDataset/Train/character_12_thaa
425
data/DevanagariHandwrittenCharacterDataset/Train/character_13_daa
425
data/DevanagariHandwrittenCharacterDataset/Train/character_14_dhaa
425
data/DevanagariHandwrittenCharacterDataset/Train/character_15_adna
425
data/DevanagariHandwrittenCharacterDataset/Train/character_16_tabala
425
data/DevanagariHandwrittenCharacterDataset/Train/character_17_tha
425
data/DevanagariHandwrittenCharacterDataset/Train/character_18_da
425
data/DevanagariHandwrittenCharacterDataset/Train/character_19_dha
425
data/DevanagariHandwrittenCharacterDataset/Train/character_1_ka
425
data/DevanagariHandwrittenCharacterDataset/Train/character_20_na
425
data/DevanagariHandwrittenCharacterDataset/Train/character_21_pa
425
data/DevanagariHandwrittenCharact

In [25]:
cv_df

,Image_Category,Category
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",character_10_yna
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",character_10_yna
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",character_10_yna
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",character_10_yna
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",character_10_yna
5,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",character_10_yna
6,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",character_10_yna
7,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",character_10_yna
8,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",character_10_yna
9,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",character_10_yna


In [18]:
def showImg(image):

    # Create figure and make it wider than the default
    fig = vv.figure()
    #fig.position.w = 700

    # Create first axes
    #a1 = vv.subplot(121)

    # Display an image
    texture2d = vv.imshow(image)
    texture2d.interpolate = True # if False the pixels are visible when zooming in

    # Enter main loop
    app = vv.use() # let visvis chose a backend for me
    app.Run()

In [19]:
cv_df.iloc[18]

Image_Category    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
Category                                           character_19_dha
Name: 18, dtype: object

In [25]:
filtered_image = getImageFromImgCat(cv_df['Image_Category'].iloc[18])
showImg(filtered_image)

In [24]:
getCategoryFromImgCat(cv_df['Image_Category'].iloc[18])

9